In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Allegheny_County', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df2 = pd.DataFrame()
df.head(10)

,cid,createDate,createBy,dataFrom,CRASH_CRN,DISTRICT,CRASH_COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,...,LANE_COUNT,RDWY_ORIENT,ROAD_OWNER,ROUTE,SPEED_LIMIT,SEGMENT,OFFSET,STREET_NAME,TOT_INJ_COUNT,SCHOOL_BUS_UNIT
0,1000000001,2020-01-28 04:38:27,David,Allegheny_County,CRASH_CRN,DISTRICT,CRASH_COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,...,LANE_COUNT,RDWY_ORIENT,ROAD_OWNER,ROUTE,SPEED_LIMIT,SEGMENT,OFFSET,STREET_NAME,TOT_INJ_COUNT,SCHOOL_BUS_UNIT\r
1,1000000002,2020-01-28 04:38:27,David,Allegheny_County,2004000001,11,02,02454,02454,2004,...,2,E,2,2032,35,0010,1000,LEWIS RUN RD,NaN,NaN
2,1000000003,2020-01-28 04:38:27,David,Allegheny_County,2004000010,11,02,02440,02440,2004,...,2,W,3,NaN,35,NaN,NaN,COAL VALLEY RD,NaN,NaN
3,1000000004,2020-01-28 04:38:27,David,Allegheny_County,2004000016,11,02,02203,02203,2004,...,2,W,2,3082,35,NaN,NaN,CLINTON FRANKFORT RD,NaN,NaN
4,1000000005,2020-01-28 04:38:27,David,Allegheny_County,2004000022,11,02,02404,02404,2004,...,2,N,4,NaN,25,NaN,NaN,NORTH BALPH AV,NaN,NaN
5,1000000006,2020-01-28 04:38:27,David,Allegheny_County,2004000030,11,02,02419,02419,2004,...,2,S,4,NaN,20,NaN,NaN,THORNBERRY DR,NaN,NaN
6,1000000007,2020-01-28 04:38:27,David,Allegheny_County,2004000043,11,02,02111,02111,2004,...,3,N,4,NaN,25,NaN,NaN,WARREN DR,NaN,NaN
7,1000000008,2020-01-28 04:38:27,David,Allegheny_County,2004000047,11,02,02435,02435,2004,...,3,E,4,NaN,30,NaN,NaN,POPLAR ST,NaN,NaN
8,1000000009,2020-01-28 04:38:27,David,Allegheny_County,2004000058,11,02,02440,02440,2004,...,4,N,2,0051,40,NaN,NaN,NaN,NaN,NaN
9,1000000010,2020-01-28 04:38:27,David,Allegheny_County,2004000059,11,02,02454,02454,2004,...,4,N,2,0051,40,NaN,NaN,CLAIRTON BL,NaN,NaN


In [4]:
df.shape

(182783, 194)

In [5]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'CRASH_CRN', 'DISTRICT',
       'CRASH_COUNTY', 'MUNICIPALITY', 'POLICE_AGCY', 'CRASH_YEAR',
       ...
       'LANE_COUNT', 'RDWY_ORIENT', 'ROAD_OWNER', 'ROUTE', 'SPEED_LIMIT',
       'SEGMENT', 'OFFSET', 'STREET_NAME', 'TOT_INJ_COUNT', 'SCHOOL_BUS_UNIT'],
      dtype='object', length=194)

In [6]:
df = df.drop(0,axis=0)

In [7]:
df2['city'] = df['dataFrom']

df2['cid'] = df['cid']

df2['injured_count'] = df['TOT_INJ_COUNT']

df2.injured_count.fillna(0,inplace=True)

In [8]:
df['CRASH_DATE'] = pd.to_datetime(df['CRASH_YEAR'] +'-'+ df['CRASH_MONTH']+'-01')

In [9]:
df['TIME_OF_DAY'].fillna('0000',inplace=True)

df['TIME_OF_DAY'].replace({'9999':'0000'},inplace=True)

df['TIME_OF_DAY'] = pd.to_datetime(df['TIME_OF_DAY'],format='%H%M')

In [10]:
df2['crash_time'] = df['CRASH_DATE'].dt.strftime('%Y-%m-%d') + ' ' + df['TIME_OF_DAY'].dt.strftime('%H:%M')
df2['crash_time'] = pd.to_datetime(df2['crash_time'])

In [11]:
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [12]:
df['LATITUDE']

1                  NaN
2         40 19:05.530
3         40 29:18.923
4                  NaN
5                  NaN
              ...     
182778    40 23:35.759
182779    40 29:29.983
182780    40 28:25.736
182781    40 32:38.310
182782    40 28:44.067
Name: LATITUDE, Length: 182782, dtype: object

In [13]:
#df2['latitude'] = df['LATITUDE'].astype('float')
#df2['longitude'] = df['LONGITUDE'].astype('float')
df2['street_name'] = df['STREET_NAME']

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182782 entries, 1 to 182782
Data columns (total 6 columns):
city             182782 non-null object
cid              182782 non-null int64
injured_count    182782 non-null object
crash_time       182782 non-null datetime64[ns]
crash_date       182782 non-null datetime64[ns]
street_name      170491 non-null object
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 9.8+ MB


In [15]:
df2.head()

,city,cid,injured_count,crash_time,crash_date,street_name
1,Allegheny_County,1000000002,0,2004-01-01 02:57:00,2004-01-01,LEWIS RUN RD
2,Allegheny_County,1000000003,0,2004-01-01 10:27:00,2004-01-01,COAL VALLEY RD
3,Allegheny_County,1000000004,0,2004-01-01 18:39:00,2004-01-01,CLINTON FRANKFORT RD
4,Allegheny_County,1000000005,0,2004-01-01 09:14:00,2004-01-01,NORTH BALPH AV
5,Allegheny_County,1000000006,0,2004-01-01 22:30:00,2004-01-01,THORNBERRY DR


In [ ]:
df2.to_sql('cleaned_crash_Allegheny_County',con = db_connection, if_exists="replace",index=False)
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)